# Experiment 3 - Call a scheduling function

Objective: Demonstrate how to use OpenAI API functions to call a scheduling API.

## Setup, imports, logging, keys

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
import openai
from datetime import datetime, timedelta

In [ ]:
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.environ['OPENAI_API_KEY']

## Step 1 - Show the type of answer that you would get without functions

If we just make a request asking to schedule an appointment, the LLM will say that it can't do that.

In [28]:
user_prompt = "I would like to schedule an appintment with a customer \
service representative for October 22 2023 at 11:00am"

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {
            "role": "user",
            "content": "I would like to schedule an appintment with a customer service representative?",
        },
    ],
)

output = completion.choices[0].message.content
print(output)

I'm sorry, but I am an AI language model and cannot schedule appointments. However, if you need assistance or have any questions, I am here to help.


## Step 2 -- Let's add a function definition and ask the same question

The response we get back wll be fundamentally different as you can see the message comes back without content but with a function_call element instead with the function name and its params.

In [30]:
function_descriptions = [
    {
        "name": "schedule_appointment",
        "description": "Schedule a new appointment with customer service representative.",
        "parameters": {
            "type": "object",
            "properties": {
                "appointment_date": {
                    "type": "string",
                    "description": "The date to schedule the appointment on",
                },
                "appointment_time": {
                    "type": "string",
                    "description": "The desired time of the appointment.",
                },
                "appointment_type": {
                    "type": "string",
                    "enum": ["human-representative", "documentation-review"],
                    "description": "The type of appointment being scheduled.",
                },
            },
            "required": ["appointment_date", "appointment_type"],
        },
    }
]

In [31]:
user_prompt = "I would like to schedule an appintment with a customer \
service representative for October 22 2023 at 11:00am"

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": user_prompt}],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",  # specify the function call
)

# It automatically fills the arguments with correct info based on the prompt
# Note: the function does not exist yet

output = completion.choices[0].message
print(output)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "schedule_appointment",
    "arguments": "{\n  \"appointment_date\": \"2023-10-22\",\n  \"appointment_time\": \"11:00am\",\n  \"appointment_type\": \"human-representative\"\n}"
  }
}


Below we actually define the function that will be called and we get the parameters from the function, and we manually call the function.

In [ ]:
def schedule_appointment(appointment_date, appointment_time, appointment_type):
    """Schedule an appointment to speak with a representative."""

    # Example output returned from an API or database
    appointment_info = {
        "appointment_date": appointment_date,
        "appointment_time": appointment_time,
        "appointment_type": appointment_type,
        "representative": "Miguel"
    }

    return json.dumps(appointment_info)


# Use the LLM output to manually call the function
# The json.loads function converts the string to a Python object
params = json.loads(output.function_call.arguments)
type(params)
print(params)



In [ ]:
# Call the function with arguments

chosen_function = eval(output.function_call.name)
appointment = chosen_function(**params)

print(appointment)



## Step 3 - LLM responds with function result

Now that we have the output from the function, we can call the LLM to now come up with a nice english message with the response that contains the function result. 

In [34]:
# --------------------------------------------------------------
# Add function result to the prompt for a final answer
# --------------------------------------------------------------

# The key is to add the function output back to the messages with role: function
second_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": user_prompt},
        {"role": "function", "name": output.function_call.name, "content": appointment},
    ],
    functions=function_descriptions,
)
response = second_completion.choices[0].message.content
print(response)

Your appointment with a customer service representative has been scheduled for October 22, 2023, at 11:00 am. Please note that the representative assigned to your appointment is Miguel.
